In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np 
from tensorflow.keras import layers
from tensorflow import keras
from datetime import datetime
import pandas as pd
import os
from datetime import datetime


2023-01-02 00:29:45.998116: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
tf.get_logger().setLevel('ERROR')
tf.autograph.set_verbosity(0)
tf.set_seed = 42
epoch = 10
max_vocab_length = 10000 # max number of words to have in our vocabulary
max_length = 100


In [14]:

# import data
train_data, test_data = tf.keras.datasets.reuters.load_data(num_words=max_vocab_length)

"""
# Check the data shape
print(
    f"Train_features : {train_features.shape} {train_features.dtype} \n" 
    f"Train_labels : {train_labels.shape} {train_labels.dtype} \n" 
    f"Test features : {test_features.shape} {test_features.dtype} \n" 
    f"Test_labels : {test_labels.shape} {test_labels.dtype} "
    )
"""
print(train_data[1])
print( test_data[1])

[ 3  4  3 ... 25  3 25]
[ 3 10  1 ...  3  3 24]


In [21]:
train_feat_data, train_labels_data = train_data
train_feat_data_a = train_feat_data 
train_feat_data_a

array([list([1, 2, 2, 8, 43, 10, 447, 5, 25, 207, 270, 5, 3095, 111, 16, 369, 186, 90, 67, 7, 89, 5, 19, 102, 6, 19, 124, 15, 90, 67, 84, 22, 482, 26, 7, 48, 4, 49, 8, 864, 39, 209, 154, 6, 151, 6, 83, 11, 15, 22, 155, 11, 15, 7, 48, 9, 4579, 1005, 504, 6, 258, 6, 272, 11, 15, 22, 134, 44, 11, 15, 16, 8, 197, 1245, 90, 67, 52, 29, 209, 30, 32, 132, 6, 109, 15, 17, 12]),
       list([1, 3267, 699, 3434, 2295, 56, 2, 7511, 9, 56, 3906, 1073, 81, 5, 1198, 57, 366, 737, 132, 20, 4093, 7, 2, 49, 2295, 2, 1037, 3267, 699, 3434, 8, 7, 10, 241, 16, 855, 129, 231, 783, 5, 4, 587, 2295, 2, 2, 775, 7, 48, 34, 191, 44, 35, 1795, 505, 17, 12]),
       list([1, 53, 12, 284, 15, 14, 272, 26, 53, 959, 32, 818, 15, 14, 272, 26, 39, 684, 70, 11, 14, 12, 3886, 18, 180, 183, 187, 70, 11, 14, 102, 32, 11, 29, 53, 44, 704, 15, 14, 19, 758, 15, 53, 959, 47, 1013, 15, 14, 19, 132, 15, 39, 965, 32, 11, 14, 147, 72, 11, 180, 183, 187, 44, 11, 14, 102, 19, 11, 123, 186, 90, 67, 960, 4, 78, 13, 68, 467, 511, 110,

In [4]:

# VECTORIZE function

def vectorize_sequences(sequences, dimension=max_vocab_length):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.
    return tf.cast(results, dtype= tf.float32)

# Vectorize and Normalize train and test to tensors with 10k columns

train_features_tf = vectorize_sequences(train_features)
test_features_tf = vectorize_sequences(test_features)

print("train_features ", train_features_tf.shape)
print("test_Features ", test_features_tf.shape)


train_features  (8982, 10000)
test_Features  (2246, 10000)


In [5]:

# ONE HOT ENCODER of the labels

one_hot_train_labels = tf.keras.utils.to_categorical(train_labels)
one_hot_test_labels = tf.keras.utils.to_categorical(test_labels)

print("one_hot_train_labels ", one_hot_train_labels.shape)
print("one_hot_test_labels ", one_hot_test_labels.shape)


one_hot_train_labels  (8982, 46)
one_hot_test_labels  (2246, 46)


In [6]:

train_dataset = tf.data.Dataset.from_tensor_slices((train_features_tf, one_hot_train_labels))
train_dataset =  train_dataset.shuffle(8982).batch(32).cache().prefetch(tf.data.AUTOTUNE)
valid_dataset = tf.data.Dataset.from_tensor_slices((test_features_tf,one_hot_test_labels))
valid_dataset = valid_dataset.batch(32).cache().prefetch(tf.data.AUTOTUNE)
print(f"Train : {train_dataset} \n"
      f"Test : {valid_dataset}")


Train : <PrefetchDataset element_spec=(TensorSpec(shape=(None, 10000), dtype=tf.float32, name=None), TensorSpec(shape=(None, 46), dtype=tf.float32, name=None))> 
Test : <PrefetchDataset element_spec=(TensorSpec(shape=(None, 10000), dtype=tf.float32, name=None), TensorSpec(shape=(None, 46), dtype=tf.float32, name=None))>


In [7]:

# Callbacks
early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_loss", # watch the val loss metric
                                                  patience=5) # if val loss decreases for 3 epochs in a row, stop training

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss",  
                                                 factor=0.2, # multiply the learning rate by 0.2 (reduce by 5x)
                                                 patience=3,
                                                 verbose=1, # print out when learning rate goes down 
                                                 min_lr=1e-7)


In [8]:

# Set random seed and create embedding layer (new embedding layer for each model)
embedding_layers = layers.Embedding(input_dim=max_vocab_length,
                                     output_dim=5,
                                     embeddings_initializer="uniform",
                                     input_length = max_length,
                                     name="embedding_layers")


In [9]:

# Build a Bidirectional RNN in TensorFlow
inputs = layers.Input(shape=(max_vocab_length,))
x = embedding_layers(inputs)
x = layers.Conv1D(32, kernel_size=5, padding="same", activation="relu")(x)
x = layers.Conv1D(64, kernel_size=5, padding="same", activation="relu")(x)
x = layers.Conv1D(128, kernel_size=5, padding="same", activation="relu")(x)
x = layers.GlobalAveragePooling1D()(x) # condense the output of our feature vector
outputs = layers.Dense(46, activation="softmax")(x)
model= tf.keras.Model(inputs, outputs, name="model_4_Bidirectional")

# Compile
model.compile(loss="categorical_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

# Get a summary of our bidirectional model
model.summary()


Model: "model_4_Bidirectional"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 10000)]           0         
                                                                 
 embedding_layers (Embedding  (None, 10000, 5)         50000     
 )                                                               
                                                                 
 conv1d (Conv1D)             (None, 10000, 32)         832       
                                                                 
 conv1d_1 (Conv1D)           (None, 10000, 64)         10304     
                                                                 
 conv1d_2 (Conv1D)           (None, 10000, 128)        41088     
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                      

In [ ]:

start = datetime.now()
# Fit the model (takes longer because of the bidirectional layers)
model_history = model.fit(train_dataset,
                           epochs=5,
                           validation_data=valid_dataset,
                           callbacks=[early_stopping, reduce_lr])

end = datetime.now()
print(f"The time taken to fit the modle is {end - start}")


In [ ]:
model.evaluate(valid_dataset)